In [ ]:
import numpy as np
import pandas as pd
import csv
import json
from gensim.models import Word2Vec
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from numpy.linalg import norm

In [ ]:
# Preprocess
ps = PorterStemmer()


with open("/content/sample_data/stopwords_new.txt") as file:
    stop = file.read()
stopwords = stop.split('\n')

num = list(range(3000))
num = [str(x) for x in num]
stopwords = stopwords + num

def removeStopwords(text):
    for word in text:
        if word in stopwords:
            text.remove(word)
    return text

def tokenize(text):
    token = [] #store token
    sub = "" # store temporary str
    for i in text:
        if i !=" ":
            if i in (",",".","'","?",'"',"@","-","$","!","_","`","(","","*","&","/",")","","''"):
                continue
            sub+=i
            if sub == "\n": #\n 是txt的文字格式，在處理token時就刪除
                sub=""
                continue
        else:
            token.append(sub)
            sub=""
    if sub :
        token.append(sub)
    return token

def preprocessing(text):
#     tokens = [word for word in nltk.word_tokenize(text)]
    tokens = tokenize(text)
    stems = [ps.stem(item) for item in tokens]
    final = []
    return stems

In [ ]:
with open("/content/sample_data/textWords.csv", 'r') as f:
    reader = csv.reader(f)
    textWords = np.array([row for row in reader], dtype=object)

In [ ]:
def cosineSim(x, y):
    x_ = x/norm(x)
    y_ = y/norm(y)
    return(np.dot(x_, y_))

### word2vec hyperparameter  
window:  
* A smaller window value considers only nearby words and captures more local context.  
* A larger window value considers words further away and captures more global context.

In [ ]:
# train word2vec
model = Word2Vec(sentences=textWords, vector_size=50, window=10, min_count=1, workers=4, epochs=100)
# model.save("word2vec_1.model")
# model = Word2Vec.load("word2vec_1.model")

# load model: can be trained continually
# model = Word2Vec.load("word2vec.model")
word2vec_words = list(model.wv.key_to_index.keys()) # keys in word2vec model

In [ ]:
# read in tfidf
with open("/content/sample_data/tfidfOverview.csv", 'r') as f:
    reader = csv.reader(f)
    tfidfOverview = np.array([list(map(float, row)) for row in reader])

# read in dictionary
with open("/content/sample_data/dictionary.txt", 'r') as f:
    dictionary = [line.strip() for line in f.readlines()]

In [ ]:
# get aggregated vectors
word2vec = []
for doc in (tfidfOverview):
    vectors = np.array([0.0 for _ in range(50)])
    for ind, val in enumerate(doc):
        if (val and dictionary[ind] in word2vec_words):
            vectors += val*model.wv[dictionary[ind]]
    word2vec.append(vectors)

### Query testing

In [ ]:
%%time
query = input()
query = tokenize(query)
query_vector = np.array([0.0 for _ in range(50)])
for word in query:
    if word in word2vec_words:
        query_vector  = np.add(model.wv[word], query_vector)

war
CPU times: user 57.2 ms, sys: 6.58 ms, total: 63.8 ms
Wall time: 3.07 s


In [ ]:
%%time
result = []
for ind in range(1000):
    result.append([cosineSim(query_vector, word2vec[ind]), ind])
sorted_result = sorted(result, key=lambda x: x[0], reverse=True)

CPU times: user 14.9 ms, sys: 0 ns, total: 14.9 ms
Wall time: 15 ms


In [ ]:
series_titles = []
with open("/content/sample_data/imdb_top_1000.csv", 'r', encoding='utf-8') as f:
    reader = csv.reader(f)
    next(reader)
    for row in reader:
        # 將 Series_Title (假設在第一列) 加入列表
        series_titles.append(row[1])

returned_movies = []
for index, item in enumerate(sorted_result):
    if index >= 5:
        break
    if(sorted_result[index][0]>0):
        movie_name = series_titles[item[1]]
        movie_Correlation = sorted_result[index][0]
        returned_movies.append(movie_name)
        print(movie_name)


The Great Escape
Hacksaw Ridge
All Quiet on the Western Front
Patton
The Pianist


In [ ]:
# for checking movie
df = pd.read_csv('/content/sample_data/imdb_top_1000_preprocessing.csv')
df_movie = df[["Series_Title", "Overview"]]

In [ ]:
for i, movie in enumerate(returned_movies):
  print(i, ". ", df_movie.iloc[df.index[df_movie.Series_Title == movie][0]][1])

0 .  Allied prisoners of war plan for several hundred of their number to escape from a German camp during World War II.
1 .  World War II American Army Medic Desmond T. Doss, who served during the Battle of Okinawa, refuses to kill people, and becomes the first man in American history to receive the Medal of Honor without firing a shot.
2 .  A German youth eagerly enters World War I, but his enthusiasm wanes as he gets a firsthand view of the horror.
3 .  The World War II phase of the career of controversial American general George S. Patton.
4 .  A Polish Jewish musician struggles to survive the destruction of the Warsaw ghetto of World War II.


In [ ]:
df_movie.iloc[df.index[df_movie.Series_Title == "Wonder"][0]][1]

'Based on the New York Times bestseller, this movie tells the incredibly inspiring and heartwarming story of August Pullman, a boy with facial differences who enters the fifth grade, attending a mainstream elementary school for the first time.'

In [ ]:
# K = int(input()) # check movie
# print("Series_Title: ", df_movie.iloc[K][0])
# print("Overview: ", df_movie.iloc[K][1])